In [1]:
se[import pandas as pd
   

In [12]:
data = pd.read_csv("./wine_quality.csv",sep="delimiter",header=None,engine='python')
data

,0
0,@relation wine-weka.filters.unsupervised.insta...
1,@attribute fixedacid numeric
2,@attribute volacid numeric
3,@attribute citricacid numeric
4,@attribute residualsugar numeric
5,@attribute chlorides numeric
6,@attribute freesulfur numeric
7,@attribute totalsulfur numeric
8,@attribute density numeric
9,@attribute pH numeric
